##### Copyright 2025 Patrick Loeber, Google LLC

In [ ]:

#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Workshop: Build with Gemini (Part 3)

<a target="_blank" href="https://colab.research.google.com/github/patrickloeber/workshop-build-with-gemini/blob/cloud-summit-nordics/03-thinking-and-tools.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This workshop teaches how to build with Gemini using the Gemini API and Python SDK.

Course outline:

- **[Part1: Quickstart + Text prompting](https://github.com/patrickloeber/workshop-build-with-gemini/blobcloud-summit-nordics/01-text-prompting.ipynb)**

- **[Part 2: Multimodal understanding (image, video, audio, docs, code)](https://github.com/patrickloeber/workshop-build-with-gemini/blob/cloud-summit-nordics/02-multimodal-understanding.ipynb)**

- **Part 3 (this notebook): Thinking models + agentic capabilities (tool usage)**
  - Thinking models
  - Structured outputps
  - Function calling
  - Code execution
  - Grounding with Google Search
  - URL Context
  - Final excercise: Give Gemini access to the PokéAPI to answer Pokémon questions

## 0. Use the Google AI Studio as playground

Explore and play with all models in the [Google AI Studio](https://aistudio.google.com/apikey).

## 1. Setup

Get a free API key in the [Google AI Studio](https://aistudio.google.com/apikey) and set up the [Google Gen AI Python SDK](https://github.com/googleapis/python-genai)

In [ ]:
%pip install -U -q google-genai

In [ ]:
from google import genai
from google.genai import types
import os
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
else:
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')


client = genai.Client(api_key=GEMINI_API_KEY)

MODEL = "gemini-2.5-flash-preview-05-20" # "gemini-2.5-pro-preview-06-05"

## Thinking models

Starting with Gemini 2.5, all models have thinking capabilities. These models use an internal "thinking process" during response generation. This process contributes to their improved reasoning capabilities and allows them to solve complex tasks, particularly complex problems in code, math, and STEM, as well as analyzing large datasets, codebases, and documents.

Thinking models are also great at working with tools to perform actions beyond generating text. This allows them to interact with external systems, execute code, or access real-time information, incorporating the results into their reasoning and final response.

(Note: Tools are also available with Gemini 2.0 models)

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents="If it takes 5 minutes to boil one egg, how long does it take to boil three eggs?"
)

print(response.text)

Thinking budgets

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents="Provide a list of 3 famous physicists and their key contributions",
    config={"thinking_config": {"thinking_budget": 1024}}
)

print(response.text)

In [ ]:
response = client.models.generate_content(
  model=MODEL,
  contents="What is the sum of the first 10 prime numbers?",
  config={"thinking_config": {"include_thoughts": True}}
)

print(response.text)

for part in response.candidates[0].content.parts:
    if part.thought:
        print("Thought summary:")
        print(part.text)

## Structured output

Gemini generates unstructured text by default, but some applications require structured text. For these use cases, you can constrain Gemini to respond with JSON, a structured data format suitable for automated processing. You can also constrain the model to respond with one of the options specified in an enum.

In [ ]:
from pydantic import BaseModel

class Recipe(BaseModel):
    recipe_name: str
    ingredients: list[str]
    prep_time_minutes: int

class RecipeList(BaseModel):
    recipes: list[Recipe]

# Using Pydantic models for structured output
response = client.models.generate_content(
    model=MODEL,
    contents="Give me 2 popular cookie recipes with ingredients and prep details.",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=RecipeList,
    ),
)

# Get structured data directly
recipes: list[Recipe] = response.parsed
for recipe in recipes.recipes:
    print(f"Recipe: {recipe.recipe_name}")
    print(f"Ingredients: {recipe.ingredients}")
    print(f"Prep Time: {recipe.prep_time_minutes} minutes")

## Function calling

Function calling lets you connect models to external tools and APIs. Instead of generating text responses, the model understands when to call specific functions and provides the necessary parameters to execute real-world actions.

In [ ]:
def get_weather(location: str) -> dict:
    """Gets current weather for a location.
    
    Args:
        location: The city name, e.g. "San Francisco"
        
    Returns:
        Weather information dictionary
    """
    # Mock weather data - in real use, you'd call a weather API
    weather_data = {
        "temperature": 22,
        "condition": "sunny", 
        "humidity": 60,
        "location": location,
        "feels_like": 24
    }
    print(f"🌤️ FUNCTION CALLED: get_weather(location='{location}')")
    return weather_data

# Define function declarations for the model
weather_function = {
    "name": "get_weather",
    "description": "Gets current weather for a location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name"
            }
        },
        "required": ["location"]
    }
}

In [ ]:
from google.genai.types import Tool

function_tool = Tool(function_declarations=[weather_function])

# Define user prompt
contents = [
    types.Content(
        role="user", parts=[types.Part(text="Whats the weather in Tokyo?")]
    )
]

# Send request with function declarations
response = client.models.generate_content(
    model=MODEL,
    contents=contents,
    config={"tools":[function_tool]}
)

# Check for function calls
function_call = response.candidates[0].content.parts[0].function_call
print(f"Model wants to call: {function_call.name}")
print(f"With arguments: {dict(function_call.args)}")

In [ ]:
# Execute the function
if function_call.name == "get_weather":
    result = get_weather(**function_call.args)
else:
    result = {"error": "Unknown function"}

print(f"Function result: {result}")

# Send function result back to model
function_response_part = types.Part.from_function_response(
    name=function_call.name,
    response={"result": result}
)
# Append function call and result of the function execution to contents
contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) # Append the model's function call message
contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

# Get final response
final_response = client.models.generate_content(
    model=MODEL,
    contents=contents,
    config={"tools":[function_tool]}
)

print(f"\nFinal response: {final_response.text}")

### Automatic Function Calling (Python Only)

When using the Python SDK, you can provide Python functions directly as tools.

The SDK handles the function call and returns the final text.

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents="What's the temperature in Boston?",
    config={
        "tools":[get_weather],
        # to diable automatic funtion calling, you can set this:
        # "automatic_function_calling": { "disable": True }
    }
)

print(response.text)

Check the function calling history:

In [ ]:
for content in response.automatic_function_calling_history:
    for part in content.parts:
        if part.function_call:
            print(part.function_call)

## Code execution

The code execution feature enables the model to generate and run Python code and learn iteratively from the results until it arrives at a final output. You can use this code execution capability to build applications that benefit from code-based reasoning and that produce text output. For example, you could use code execution in an application that solves equations or processes text.

In [ ]:
from google.genai.types import Tool, ToolCodeExecution

code_tool = Tool(code_execution=ToolCodeExecution)

# In your prompt, give instruction to use/generate code
response = client.models.generate_content(
  model=MODEL,
  contents='Create a bar chart showing the population of the 5 largest cities in the world. Generate code to solve it. Use matplotlib.',
  config={"tools":[code_tool]}
)


from IPython.display import Image, Markdown, HTML
for part in response.candidates[0].content.parts:
    if part.text is not None:
        display(Markdown(part.text))
    if part.executable_code is not None:
        display(Markdown(f"```python\n{part.executable_code.code}\n```"))
    if part.code_execution_result is not None:
        display(Markdown(f"#### Output\n{part.code_execution_result.output}"))
    if part.inline_data is not None:
        display(Image(data=part.inline_data.data, width=800, format="png"))

## Grounding with Google Search

If Google Search is configured as a tool, Gemini can decide when to use Google Search to improve the accuracy and recency of responses.

Here's a question about a recent event without Google Search:



In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents="Who won the super bowl in 2025?",
)

print(response.text)

In [ ]:
from google.genai.types import Tool, GoogleSearch

google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model=MODEL,
    contents="Who won the super bowl in 2025?",
    config={"tools":[google_search_tool]}
)

print(response.text)

In [ ]:
# To get grounding metadata as web content.
HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

## URL Context Tool

In [ ]:
from google.genai.types import Tool, UrlContext

url_context_tool = Tool(url_context=UrlContext())

response = client.models.generate_content(
    model=MODEL,
    contents="Summarize the key features and benefits mentioned on https://www.python.org/about/ in 3 bullet points.",
    config={"tools":[url_context_tool]}
)

print("🌐 Python.org Summary:")
print(response.text)

## !! Exercise !!: Get Pokémon stats

- Define a function that can work with the PokéAPI and get Pokémon stats.
- Endpoint to use: `GET https://pokeapi.co/api/v2/pokemon/<pokekon_name>`
- Call Gemini and give it access to the function, then answer questions like: `"What stats does the Pokemon Squirtle have?"`


In [ ]:
import requests

def get_pokemon_info(pokemon: str) -> dict:
    """Gets pokemon info for a given pokemon name.

    Args:
        pokemon: The name of the pokemon.

    Returns:
        A dictionary containing the info.
    """
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon.lower()}"
    # TODO: send GET request to endpoint and return JSON


# Configure the function call
response = client.models.generate_content(
    model=MODEL,
    contents="What stats does the Pokemon Squirtle have?",
    config=...
)

print(response.text)

In [ ]:
for content in response.automatic_function_calling_history:
    for part in content.parts:
        if part.function_call:
            print(part.function_call)

## Recap & Next steps

Awesome work! You learned about thinking models with advanced reasoning capabilities and how to combine Gemini with tools for agentic use cases.

More helpful resources:

- [Thinking docs](https://ai.google.dev/gemini-api/docs/thinking)
- [Structured output docs](https://ai.google.dev/gemini-api/docs/structured-output?lang=python)
- [Code execution docs](https://ai.google.dev/gemini-api/docs/code-execution?lang=python)
- [Grounding docs](https://ai.google.dev/gemini-api/docs/grounding?lang=python)
- [Function calling docs](https://ai.google.dev/gemini-api/docs/function-calling?example=weather)
- [URL context docs](https://ai.google.dev/gemini-api/docs/url-context)

🎉🎉**Conratulations, you completed the workshop!**🎉🎉

**Next steps**: There's even more you can do with Gemini which we didn't cover in this workshop:

- **[Part 4: Try the Live API](https://github.com/patrickloeber/workshop-build-with-gemini/blob/cloud-summit-nordics/04-live-api)**
- **[Part 5: Use MCP with Gemini](https://github.com/patrickloeber/workshop-build-with-gemini/blob/cloud-summit-nordics/05-mcp)**
